# 高性能Pandas：eval()与query() (High-Performance Pandas: eval() and query())

前面的章节已经介绍过，Python 数据科学生态环境的强大力量建立在NumPy 与Pandas 的 基础之上，并通过直观的语法将基本操作转换成C 语言：在NumPy 里是向量化/ 广播运 算，在Pandas 里是分组型的运算。虽然这些抽象功能可以简洁高效地解决许多问题，但是 它们经常需要创建临时中间对象，这样就会占用大量的计算时间与内存。

Pandas 从0.13 版开始（2014 年1 月）就引入了实验性工具，让用户可以直接运行C 语言 速度的操作，不需要十分费力地配置中间数组。它们就是eval() 和query() 函数，都依赖 于Numexpr (https://github.com/pydata/numexpr) 程序包。我们将在下面的Notebook 中演示 其用法，并介绍一些使用时的注意事项。

In [1]:
import numpy as np
import pandas as pd

## query()与eval()的设计动机：复合代数式(Motivating query() and eval(): Compound Expressions)
前面已经介绍过，NumPy 与Pandas 都支持快速的向量化运算。例如，你可以对下面两个数组进行求和：

In [2]:
rng = np.random.RandomState(42)
x = rng.rand(10**6)
y = rng.rand(10**6)
%time x + y

Wall time: 3.95 ms


array([0.96969574, 1.31543145, 0.73736956, ..., 1.1011828 , 0.76733033,
       1.62106496])

这样做比普通的Python 循环或列表综合要快很多：

In [3]:
%time np.fromiter((xi + yi for xi, yi in zip(x, y)), dtype=x.dtype, count=len(x))

Wall time: 204 ms


array([0.96969574, 1.31543145, 0.73736956, ..., 1.1011828 , 0.76733033,
       1.62106496])

但是这种运算在处理复合代数式（compound expression）问题时的效率比较低，例如下面的表达式：

In [4]:
mask = (x > 0.5) & (y < 0.5)

由于NumPy 会计算每一个代数子式，因此这个计算过程等价于：

In [5]:
tmp1 = (x > 0.5)
tmp2 = (y < 0.5)
mask = tmp1 & tmp2

也就是说，每段中间过程都需要显式地分配内存。如果x 数组和y 数组非常大，这么运算 就会占用大量的时间和内存消耗。Numexpr 程序库可以让你在不为中间过程分配全部内 存的前提下，完成元素到元素的复合代数式运算。虽然Numexpr 文档（https://github.com/ pydata/numexpr）里提供了更详细的内容，但是简单点儿说，这个程序库其实就是用一个 NumPy 风格的字符串代数式进行运算：

In [6]:
import numexpr
mask_numexpr = numexpr.evaluate('(x > 0.5) & (y < 0.5)')
np.allclose(mask, mask_numexpr)

True

这么做的好处是，由于Numexpr 在计算代数式时不需要为临时数组分配全部内存，因此计 算比NumPy 更高效，尤其适合处理大型数组。马上要介绍的Pandas 的eval() 和query() 工具其实也是基于Numexpr 实现的。

## 用pandas.eval()实现高性能运算(pandas.eval() for Efficient Operations)
Pandas 的eval() 函数用字符串代数式实现了DataFrame 的高性能运算， 例如下面的DataFrame：

In [7]:
nrows, ncols = 100000, 100
rng = np.random.RandomState(42)
df1, df2, df3, df4 = (pd.DataFrame(rng.rand(nrows, ncols))
                      for i in range(4))

如果要用普通的Pandas 方法计算四个DataFrame 的和，可以这么写：

In [8]:
%time df5 = df1 + df2 + df3 + df4

Wall time: 63.8 ms


也可以通过pd.eval 和字符串代数式计算并得出相同的结果：

In [9]:
%time df5 = pd.eval('df1 + df2 + df3 + df4')

Wall time: 37.9 ms


这个eval() 版本的代数式比普通方法快一倍（而且内存消耗更少），结果也是一样的：

In [10]:
np.allclose(df1 + df2 + df3 + df4,
            pd.eval('df1 + df2 + df3 + df4'))

True

#### pd.eval()支持的运算
从Pandas v0.16 版开始，pd.eval() 就支持许多运算了。为了演示这些运算，创建一个整数类型的DataFrame：

In [11]:
df1, df2, df3, df4, df5 = (pd.DataFrame(rng.randint(0, 1000, (100, 3)))
                           for i in range(5))

##### (1) 算术运算符。
pd.eval() 支持所有的算术运算符，例如：

In [12]:
result1 = -df1 * df2 / (df3 + df4) - df5
result2 = pd.eval('-df1 * df2 / (df3 + df4) - df5')
np.allclose(result1, result2)

True

##### (2)比较运算符。
pd.eval() 支持所有的比较运算符，包括链式代数式（chained expression）：

In [13]:
result1 = (df1 < df2) & (df2 <= df3) & (df3 != df4)
result2 = pd.eval('df1 < df2 <= df3 != df4')
np.allclose(result1, result2)

True

##### (3) 位运算符。
pd.eval() 支持&（与）和|（或）等位运算符：

In [14]:
result1 = (df1 < 0.5) & (df2 < 0.5) | (df3 < df4)
result2 = pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)')
np.allclose(result1, result2)

True

另外，你还可以在布尔类型的代数式中使用and 和or 等字面值：

In [15]:
result3 = pd.eval('(df1 < 0.5) and (df2 < 0.5) or (df3 < df4)')
np.allclose(result1, result3)

True

##### (4) 对象属性与索引。
pd.eval() 可以通过obj.attr 语法获取对象属性，通过obj[index] 语法获取对象索引：

In [16]:
result1 = df2.T[0] + df3.iloc[1]
result2 = pd.eval('df2.T[0] + df3.iloc[1]')
np.allclose(result1, result2)

True

##### (5) 其他运算。
目前pd.eval() 还不支持函数调用、条件语句、循环以及更复杂的运算。如果你想要进行这些运算，可以借助Numexpr 来实现。

## 用DataFrame.eval()实现列间运算(DataFrame.eval() for Column-Wise Operations)
由于pd.eval() 是Pandas 的顶层函数，因此DataFrame 有一个eval() 方法可以做类似的运算。使用eval() 方法的好处是可以借助列名称进行运算，示例如下：

In [17]:
df = pd.DataFrame(rng.rand(1000, 3), columns=['A', 'B', 'C'])
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


如果用前面介绍的pd.eval()，就可以通过下面的代数式计算这三列：

In [18]:
result1 = (df['A'] + df['B']) / (df['C'] - 1)
result2 = pd.eval("(df.A + df.B) / (df.C - 1)")
np.allclose(result1, result2)

True

而DataFrame.eval() 方法可以通过列名称实现简洁的代数式：

In [19]:
result3 = df.eval('(A + B) / (C - 1)')
np.allclose(result1, result3)

True

请注意，这里用列名称作为变量来计算代数式，结果同样是正确的。

#### 1. 用DataFrame.eval()新增列
除了前面介绍的运算功能，DataFrame.eval() 还可以创建新的列。还用前面的DataFrame来演示，列名是'A'、'B' 和'C':

In [20]:
df.head()

,A,B,C
0,0.375506,0.406939,0.069938
1,0.069087,0.235615,0.154374
2,0.677945,0.433839,0.652324
3,0.264038,0.808055,0.347197
4,0.589161,0.252418,0.557789


可以用df.eval() 创建一个新的列'D'，然后赋给它其他列计算的值：

In [21]:
df.eval('D = (A + B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,11.187620
1,0.069087,0.235615,0.154374,1.973796
2,0.677945,0.433839,0.652324,1.704344
3,0.264038,0.808055,0.347197,3.087857
4,0.589161,0.252418,0.557789,1.508776


还可以修改已有的列：

In [22]:
df.eval('D = (A - B) / C', inplace=True)
df.head()

,A,B,C,D
0,0.375506,0.406939,0.069938,-0.449425
1,0.069087,0.235615,0.154374,-1.078728
2,0.677945,0.433839,0.652324,0.374209
3,0.264038,0.808055,0.347197,-1.566886
4,0.589161,0.252418,0.557789,0.603708


#### 2. DataFrame.eval()使用局部变量
DataFrame.eval() 方法还支持通过@ 符号使用Python 的局部变量，如下所示：

In [23]:
column_mean = df.mean(1)
result1 = df['A'] + column_mean
result2 = df.eval('A + @column_mean')
np.allclose(result1, result2)

True

@ 符号表示“这是一个变量名称而不是一个列名称”，从而让你灵活地用两个“命名空 间”的资源（列名称的命名空间和Python 对象的命名空间）计算代数式。需要注意的 是，@ 符号只能在DataFrame.eval() 方法中使用，而不能在pandas.eval() 函数中使用， 因为pandas.eval() 函数只能获取一个（Python）命名空间的内容。

## DataFrame.query()方法(DataFrame.query() Method)
DataFrame 基于字符串代数式的运算实现了另一个方法，被称为query()，例如：

In [24]:
result1 = df[(df.A < 0.5) & (df.B < 0.5)]
result2 = pd.eval('df[(df.A < 0.5) & (df.B < 0.5)]')
np.allclose(result1, result2)

True

和前面介绍过的DataFrame.eval() 一样，这是一个用DataFrame 列创建的代数式，但是不能用DataFrame.eval() 语法5。不过，对于这种过滤运算，你可以用query() 方法：

In [25]:
result2 = df.query('A < 0.5 and B < 0.5')
np.allclose(result1, result2)

True

In [26]:
result2 = df.query('A < 0.5 & B < 0.5')
np.allclose(result1, result2)

True

除了计算性能更优之外，这种方法的语法也比掩码代数式语法更好理解。需要注意的是，query() 方法也支持用@ 符号引用局部变量：

In [27]:
Cmean = df['C'].mean()
result1 = df[(df.A < Cmean) & (df.B < Cmean)]
result2 = df.query('A < @Cmean and B < @Cmean')
np.allclose(result1, result2)

True

## 性能决定使用时机(Performance: When to Use These Functions)
在考虑要不要用这两个函数时，需要思考两个方面：计算时间和内存消耗，而内存消耗是 更重要的影响因素。就像前面介绍的那样，每个涉及NumPy 数组或Pandas 的DataFrame 的复合代数式都会产生临时数组，例如：

In [28]:
x = df[(df.A < 0.5) & (df.B < 0.5)]

它基本等价于：

In [29]:
tmp1 = df.A < 0.5
tmp2 = df.B < 0.5
tmp3 = tmp1 & tmp2
x = df[tmp3]

如果临时DataFrame 的内存需求比你的系统内存还大（通常是几吉字节），那么最好还是使用eval() 和query() 代数式。你可以通过下面的方法大概估算一下变量的内存消耗：

In [30]:
df.values.nbytes

32000

In [31]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
A    1000 non-null float64
B    1000 non-null float64
C    1000 non-null float64
D    1000 non-null float64
dtypes: float64(4)
memory usage: 31.3 KB


在性能方面，即使你没有使用最大的系统内存，eval() 的计算速度也比普通方法快。现在的 性能瓶颈变成了临时DataFrame 与系统CPU 的L1 和L2 缓存（在2016 年依然是几兆字节） 之间的对比了——如果系统缓存足够大，那么eval() 就可以避免在不同缓存间缓慢地移动 临时文件。在实际工作中，我发现普通的计算方法与eval/ query 计算方法在计算时间上的差 异并非总是那么明显，普通方法在处理较小的数组时反而速度更快！ **eval/ query 方法的优点 主要是节省内存，有时语法也更加简洁。**

我们已经介绍了eval() 与query() 的绝大多数细节，若想了解更多的信息，请参考Pandas 文档。尤其需要注意的是，可以通过设置不同的解析器和引擎来执行这些查询，相关细节 请参考Pandas 文档中“Enhancing Performance”（http://pandas.pydata.org/pandas-docs/dev/ enhancingperf.html）节。

# eval-字符串表达式
## eval函数：
```python
pd.eval(expr, parser='pandas', engine=None, truediv=True, local_dict=None,
global_dict=None, resolvers=(), level=0, target=None, inplace=False)
df.eval(expr, inplace=False, **kwargs)
 
用途：
   # 使用各种后端将Python表达式评估为字符串。
   # 顶级函数pandas.eval()实现Series和DataFrame对象的表达式评估
 
参数：
expr：str或unicode要评估的仅仅是Python 表达式（支持除移位的运算符）
parser：字符串,默认'pandas'解析器,{'pandas','python'}
engine：string或None,默认'numexpr',{'python','numexpr'}评估表达式的引擎
truediv：bool,可选是否使用真正的除法,如Python> = 3
local_dict：dict或None,可选局部变量字典,默认情况下取自locals()
global_dict：dict或None,可选全局变量字典,默认情况下取自globals()
 
resolvers：dict-like或None列表,可选
   # 实现__getitem__对象列表,注入变量查找的其他命名空间集合。
level：int,可选 要遍历并添加到当前范围的先前堆栈帧数。
target：object,optional,default None
   分配目标对象。当表达式中存在变量赋值时使用。
target必须支持使用字符串键的项目赋值,如返回副本必须支持.copy()
 
inplace=False 返回副本或就地修改
```

## 语法：
支持的语法
```python
  算术运算,除left shift(<<)和right shift(>>)#df + 2 * pi / s ** 4 % 42
  比较操作                                  #2 < df < df2链式比较
  布尔运算                                  #df < df2 and df3 < df4 or not df_bool
  list和tuple文字 #[1, 2](1, 2)
  属性访问        #df.a
  下标表达式      #df[0]
  简单变量评估    #pd.eval('df')
  数学函数：#sin,cos,exp,log,expm1,log1p, sqrt,sinh,cosh,tanh,arcsin,
           # arccos,arctan,arccosh, arcsinh,arctanh,abs和arctan2。
不合法语法：
   其他函数,表达式,语句;布尔表达式仅包含标量值,非bool类型或标量按位操作
   is/ 操作is not;list/ set/ dict复杂构造
执行内置复杂类型的表达式用Numexpr 库
```
## 注意：
```python
1)DataFrame更有效地评估大对象;
2)底层引擎一次性评估大型算术和布尔表达式(默认numexpr评估)
  https://numexpr.readthedocs.io/en/latest/user_guide.html
3)NumPy和Pandas支持快速矢量化操作；对复合表达式每步骤需分配内存。
  Numexpr库按元素计算复合表达式,不用分配完整的中间数组。
 
使用：
  经验法则是仅eval()在拥有 DataFrame超过10,000行时才使用。
  eval() 支持引擎支持的所有算术表达式,以及仅在pandas中可用的一些扩展。
```

## 实例1：顶级函数pd.eval()支持的操作

In [32]:
df1,df2,df3,df4= (pd.DataFrame(np.random.randint(0, 10, (3, 4))) 
                  for i in range(4))

In [33]:
#实例1.1： 算术运算符
result = -df1 * df2 / (df3 + df4)
result= pd.eval('-df1 * df2 / (df3 + df4) ')#等价
 
# 实例1.2：比较运算符
pd.eval('df1 < df2 <= df3 != df4')
 
# 实例1.3：位操作符
pd.eval('(df1 < 0.5) & (df2 < 0.5) | (df3 < df4)')
pd.eval('(df1 < 0.5) and (df2 < 0.5) or (df3 < df4)')
 
# 实例1.4：对象属性和索引
pd.eval('df2.T[0] + df3.iloc[1]')
 
#============================================================
# 实例1.5：pandas.eval()解析器
# 可用两种不同的解析器和两种不同的引擎作为后端。
 
# 默认'pandas'解析器允许使用更直观的语法来表达类似查询的操作(比较,连接和析取)
# 'python'引擎通常没有用, 它有点慢
 
expr1 = '(df1 > 0) & (df2 > 0) & (df3 > 0)' # &
expr2 = 'df1 > 0 & df2 > 0 & df3 > 0'       # 无括号
expr3 = 'df1 > 0 and df2 > 0 and df3 > 0'   # and
 
np.allclose( pd.eval(expr1),pd.eval(expr3) )

True

## 实例2：df.eval()

In [34]:
# 实例2.1：按名称指定列
df= pd.DataFrame(np.random.randint(0, 10, (3, 4)),columns=list('ABCD'))
 
result = (df['A'] + df['B']) / (df['C'] - 1)
result = pd.eval("(df.A + df.B) / (df.C - 1)")
result = df.eval('(A + B) / (C - 1)')   # 求值表达式将列名看做变量
 
# 实例2.2：DataFrame.eval()中赋值
df.eval('E = (A + B) / C', inplace=True)# 创建新列D
df.eval('D = (A - B) / C', inplace=True)# 列修改
 
# 实例2.3：@本地变量 顶级函数直接使用
 
column_mean = df.mean(1)
df['A'] + column_mean
df.eval('A + @column_mean')
pd.eval('df.A + column_mean')
 
# 实例2.4：多行书写
df.eval("""
C = A + B
D = A - C
A = 1""", inplace=False)
 
# 实例2.5：局部变量名和列名相同
D=-2
df.eval('D=@D-8')

,A,B,C,D,E
0,9,6,4,-10,3.750000
1,5,5,9,-10,1.111111
2,0,0,1,-10,0.000000


## 实例3：numexpr

In [35]:
df= pd.DataFrame(np.random.randint(0, 10, (3, 4)),columns=list('ABCD'))
df

,A,B,C,D
0,7,8,4,3
1,3,2,6,5
2,7,6,2,8


In [36]:
x=df.A
y=df.B

df[(x<7) & (y>6)]                       #必须加括号
df[df.eval('A<7 and B>6')]
df.query('A<7 and B>6')
df[numexpr.evaluate('(x < 7) & (y >6)')]#所有值相同 #必须加括号

,A,B,C,D


# query-字符串表达式查询
## 函数：
```python
df.query（expr，inplace = False，** kwargs ）

参数：
# expr：str要评估的查询字符串。你可以在环境中引用变量，在它们前面添加一个'@'字符 。@a + b
# inplace=False：是否修改数据或返回副本
# kwargs：dict关键字参数
 
返回：DataFrame
 
注意：
# 默认修改Python语法'&'/'and'和'|'/'or'位运算符优先级高于布尔表达式，不同于Python
# 关键字参数parser='python'执行Python评估。
# engine='python' 用Python本身作为后端来传递评估表达式。不建议效率低。
 
# 默认实例df.index和 df.columns属性 DataFrame放在查询命名空间中，
# 这允许您将框架的索引和列视为框架中的列。标识符index用于帧索引; 
# 您还可以使用索引的名称在查询中标识它。

性能：
    # 涉及NumPy数组或Pandas DataFrames的复合表达式都会导致隐式创建临时数组
    # eval/query用在数据（df.values.nbytes>1万）性能提升明显；传统方法在小数组时运行得更快；
    # eval/query好处主要时节省内存，以及有时候简洁得语法
    # 可用指定不同解析器和引擎来运行这些查询；参见"Enhancing Performance" 。
```

## 访问列和索引
您可以获取列的值，例如，查询列b具有列值a和值c之间的值：

In [37]:
n = 10
df = pd.DataFrame(np.random.rand(n, 3), columns=list('abc'))
df

,a,b,c
0,0.568958,0.057684,0.598257
1,0.909007,0.657400,0.408223
2,0.037334,0.270458,0.566354
3,0.003324,0.066078,0.292900
4,0.650177,0.033303,0.622228
5,0.429106,0.776407,0.176048
6,0.724282,0.302647,0.420293
7,0.122396,0.884787,0.977965
8,0.770282,0.537784,0.671048
9,0.263694,0.419929,0.362316


In [38]:
df[(df.a < df.b) & (df.b < df.c)]  # pure python

,a,b,c
2,0.037334,0.270458,0.566354
3,0.003324,0.066078,0.292900
7,0.122396,0.884787,0.977965


In [39]:
df.query('(a < b) & (b < c)')  # query

,a,b,c
2,0.037334,0.270458,0.566354
3,0.003324,0.066078,0.292900
7,0.122396,0.884787,0.977965


如果没有名称的列，则访问命名索引。

In [40]:
df = pd.DataFrame(np.random.randint(n / 2, size=(n, 2)), columns=list('bc'))
df.index.name = 'a'
df.query('a < b and b < c')

,b,c
a,,


如果您不希望或不能命名索引，则可以index访问索引：

In [41]:
df = pd.DataFrame(np.random.randint(n, size=(n, 2)), columns=list('bc'))
df.query('index < b < c')

,b,c
1,3,9


**注意 如果索引的名称与列名称重叠，则列名称优先。**例如，

In [42]:
df = pd.DataFrame({'a': np.random.randint(5, size=6)})
df.index.name = 'a'
df.query('a > 2')  # uses the column 'a', not the index

,a
a,


您仍然可以使用特殊标识符'index'在查询表达式中使用索引：

In [43]:
df.query('index > 2')

,a
a,
3,0
4,2
5,2


如果由于某种原因你有一个名为列的列index，那么你也可以引用索引ilevel_0，但是此时你应该考虑将列重命名为不那么模糊的列。

## MultiIndex query()语法
You can also use the levels of a DataFrame with a MultiIndex as if they were columns in the frame:

您还可以使用的水平DataFrame带 MultiIndex，好像他们是列：

In [44]:
n = 10
colors = np.random.choice(['red', 'green'], size=n)
foods = np.random.choice(['eggs', 'ham'], size=n)
index = pd.MultiIndex.from_arrays([colors, foods], names=['color', 'food'])
df = pd.DataFrame(np.random.randn(n, 2), index=index)
df

0         1
color food                    
red   eggs  0.758455 -0.201190
green ham  -1.563642  0.586583
red   eggs  1.013607 -1.135069
      eggs  0.959583 -1.753532
green eggs  0.562048  0.142800
      ham   1.230515 -1.374338
red   ham   0.334146 -1.780690
      eggs -1.315104  0.604552
      ham   2.108900  1.151797
      eggs -0.321386  0.234052

In [45]:
df.query('color == "red"')

0         1
color food                    
red   eggs  0.758455 -0.201190
      eggs  1.013607 -1.135069
      eggs  0.959583 -1.753532
      ham   0.334146 -1.780690
      eggs -1.315104  0.604552
      ham   2.108900  1.151797
      eggs -0.321386  0.234052

If the levels of the MultiIndex are unnamed, you can refer to them using special names:

如果MultiIndex未命名，您可以使用特殊名称引用它们：

In [46]:
df.index.names = [None, None]
df

0         1
red   eggs  0.758455 -0.201190
green ham  -1.563642  0.586583
red   eggs  1.013607 -1.135069
      eggs  0.959583 -1.753532
green eggs  0.562048  0.142800
      ham   1.230515 -1.374338
red   ham   0.334146 -1.780690
      eggs -1.315104  0.604552
      ham   2.108900  1.151797
      eggs -0.321386  0.234052

In [47]:
df.query('ilevel_0 == "red"')

0         1
red eggs  0.758455 -0.201190
    eggs  1.013607 -1.135069
    eggs  0.959583 -1.753532
    ham   0.334146 -1.780690
    eggs -1.315104  0.604552
    ham   2.108900  1.151797
    eggs -0.321386  0.234052

The convention is ilevel_0, which means “index level 0” for the 0th level of the index.

约定是ilevel_0，这意味着第0级的“索引级别0” index。

## query()Python与pandas语法比较
Full numpy-like syntax:

In [48]:
df = pd.DataFrame(np.random.randint(n, size=(n, 3)), columns=list('abc'))
df.query('(a < b) & (b < c)')

,a,b,c


In [49]:
 df[(df.a < df.b) & (df.b < df.c)]

,a,b,c


Slightly nicer by removing the parentheses (by binding making comparison operators bind tighter than & and |).

通过删除括号略微更好（通过绑定使比较运算符绑定比&和更紧|）

In [50]:
df.query('a < b & b < c')

,a,b,c


Use English instead of symbols:

In [51]:
df.query('a < b and b < c')

,a,b,c


Pretty close to how you might write it on paper:

In [52]:
df.query('a < b < c')

,a,b,c


##  使用in 和 not in(The in and not in operators)

In [53]:
# get all rows where columns "a" and "b" have overlapping values
df = pd.DataFrame({'a': list('aabbccddeeff'), 'b': list('aaaabbbbcccc'),
                   'c': np.random.randint(5, size=12),
                   'd': np.random.randint(9, size=12)})
df

,a,b,c,d
0,a,a,0,6
1,a,a,4,1
2,b,a,0,3
3,b,a,2,7
4,c,b,4,8
5,c,b,4,6
6,d,b,4,6
7,d,b,0,3
8,e,c,2,5
9,e,c,3,3


In [54]:
df.query('a in b')

,a,b,c,d
0,a,a,0,6
1,a,a,4,1
2,b,a,0,3
3,b,a,2,7
4,c,b,4,8
5,c,b,4,6


In [55]:
# How you'd do it in pure Python
df[df.a.isin(df.b)]

,a,b,c,d
0,a,a,0,6
1,a,a,4,1
2,b,a,0,3
3,b,a,2,7
4,c,b,4,8
5,c,b,4,6


In [56]:
df.query('a not in b')

,a,b,c,d
6,d,b,4,6
7,d,b,0,3
8,e,c,2,5
9,e,c,3,3
10,f,c,1,1
11,f,c,0,6


In [57]:
# pure Python
df[~df.a.isin(df.b)]

,a,b,c,d
6,d,b,4,6
7,d,b,0,3
8,e,c,2,5
9,e,c,3,3
10,f,c,1,1
11,f,c,0,6


You can combine this with other expressions for very succinct queries:

您可以将此与其他表达式结合使用，以获得非常简洁的查询：

In [58]:
# rows where cols a and b have overlapping values
# and col c's values are less than col d's
df.query('a in b and c < d')

,a,b,c,d
0,a,a,0,6
2,b,a,0,3
3,b,a,2,7
4,c,b,4,8
5,c,b,4,6


In [59]:
# pure Python
df[df.a.isin(df.b) & (df.c < df.d)]

,a,b,c,d
0,a,a,0,6
2,b,a,0,3
3,b,a,2,7
4,c,b,4,8
5,c,b,4,6


In [60]:
df.b.isin(df.a)

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
Name: b, dtype: bool

## ==运算符与list对象的特殊用法(Special use of the == operator with list objects¶)
Comparing a list of values to a column using ==/!= works similarly to in/not in.

一个比较list值的使用列==/ !=，以类似in/ not in

In [61]:
df.query('b == ["a", "b", "c"]')

,a,b,c,d
0,a,a,0,6
1,a,a,4,1
2,b,a,0,3
3,b,a,2,7
4,c,b,4,8
5,c,b,4,6
6,d,b,4,6
7,d,b,0,3
8,e,c,2,5
9,e,c,3,3


In [62]:
# pure Python
df[df.b.isin(["a", "b", "c"])]

,a,b,c,d
0,a,a,0,6
1,a,a,4,1
2,b,a,0,3
3,b,a,2,7
4,c,b,4,8
5,c,b,4,6
6,d,b,4,6
7,d,b,0,3
8,e,c,2,5
9,e,c,3,3


In [63]:
df.query('b == ["a", "b", "c"]')

,a,b,c,d
0,a,a,0,6
1,a,a,4,1
2,b,a,0,3
3,b,a,2,7
4,c,b,4,8
5,c,b,4,6
6,d,b,4,6
7,d,b,0,3
8,e,c,2,5
9,e,c,3,3


In [64]:
df.query('c == [1, 2]')

,a,b,c,d
3,b,a,2,7
8,e,c,2,5
10,f,c,1,1


In [65]:
df.query('c != [1, 2]')

,a,b,c,d
0,a,a,0,6
1,a,a,4,1
2,b,a,0,3
4,c,b,4,8
5,c,b,4,6
6,d,b,4,6
7,d,b,0,3
9,e,c,3,3
11,f,c,0,6


In [66]:
# using in/not in
df.query('[1, 2] in c')

,a,b,c,d
3,b,a,2,7
8,e,c,2,5
10,f,c,1,1


In [67]:
df.query('[1, 2] not in c')

,a,b,c,d
0,a,a,0,6
1,a,a,4,1
2,b,a,0,3
4,c,b,4,8
5,c,b,4,6
6,d,b,4,6
7,d,b,0,3
9,e,c,3,3
11,f,c,0,6


In [68]:
# pure Python
df[df.c.isin([1, 2])]

,a,b,c,d
3,b,a,2,7
8,e,c,2,5
10,f,c,1,1


## 布尔运算符(Boolean Operators)
您可以使用单词not或~运算符否定布尔表达式。

In [69]:
df = pd.DataFrame(np.random.rand(n, 3), columns=list('abc'))
df['bools'] = np.random.rand(len(df)) > 0.5
df.query('~bools')

,a,b,c,bools
3,0.764120,0.170004,0.587560,False
6,0.517044,0.909547,0.886458,False


In [70]:
df.query('not bools')

,a,b,c,bools
3,0.764120,0.170004,0.587560,False
6,0.517044,0.909547,0.886458,False


In [71]:
df[df['bools']]

,a,b,c,bools
0,0.328160,0.093478,0.346545,True
1,0.462610,0.470435,0.685974,True
2,0.264906,0.297538,0.148034,True
4,0.993455,0.345318,0.281437,True
5,0.246454,0.870827,0.584801,True
7,0.967369,0.144573,0.770046,True
8,0.671812,0.897004,0.995789,True
9,0.100252,0.321905,0.050499,True


Of course, expressions can be arbitrarily complex too:

当然，表达式也可以是任意复杂的：

In [72]:
# short query syntax
shorter = df.query('a < b < c and (not bools) or bools > 2')

# equivalent in pure Python
longer = df[(df.a < df.b) & (df.b < df.c) & (~df.bools) | (df.bools > 2)]
shorter

,a,b,c,bools


In [73]:
shorter == longer

,a,b,c,bools


## query性能 (Performance of query())
DataFrame.query() using numexpr is slightly faster than Python for large frames.
![query-perf.png](imgs/query-perf.png)

Note: You will only see the performance benefits of using the numexpr engine with DataFrame.query() if your frame has more than approximately 200,000 rows.
![query-perf-small.png](imgs/query-perf-small.png)

## 实例
#### 实例1

In [74]:
df = pd.DataFrame(np.arange(12).reshape(3, 4), columns=list('ABCD'))


# 实例1.1：python,numexpr 方式比较
result1 = df[(df.A < 8) & (df.B < 9)] #python方式
result2 = pd.eval('df[(df.A < 8) & (df.B < 9)]')#numexpr 方式
 
np.allclose(result1, result2) # True
 
# 实例1.2：eval,query,比较
# 相同点：计算表达式结果
# 不同点：eval若表达式为逻辑，结果返回bool数组;query则返回bool数组的数据
 
import numexpr
 
result3= df[df.eval('A<8 & B<9')]
result4 = df.query('A < 8 and B < 9')
result3.equals(result4) #True 结果result1==result2==result3==result4
 
a=df.A
b=df.B
result5= df[numexpr.evaluate('(a<8) &(b < 9)')]#等效；表达式不能含df.A

#### 实例2

In [75]:
# 实例2：@符合来标记本地变量
Cmean = df['C'].mean() #6.0
result1 = df[(df.A < Cmean) & (df.B < Cmean)]
result1 = df.query('A < @Cmean and B < @Cmean')#等价
result1

,A,B,C,D
0,0,1,2,3
1,4,5,6,7


#### 实例3：索引

In [76]:
# 实例3.1：列名
df.index.name='a'
df.query('(A < B) & (B < C)') #numexpr 方式 A,B,C为列名
 
# 实例3.2：单索引名+列名
df.query('a < B and B < C')   #a为单索引名，B,C为列名
df.query('index < B < C')     #index为单索引(非索引名)，B,C为列名
 
# 实例3.3：单索引名a与列名a相同
df.query('a > 2')             # 用列'a',单索引名a与列名a相同列名称优先
df.query('index > 2')         #index为单索引(非索引名),单索引名a与列名a相同列名称优先
 
# 实例3.4：列名为index- 应该考虑将列重命名
#df['index'] = 1
#df.query('ilevel_0 > 2')      #ilevel_0为单索引(非索引名)
#似乎不行

,A,B,C,D
a,,,,


#### 实例4：多索引MultiIndex

In [77]:
colors = np.random.choice(['red', 'blue'], size=6)
foods = np.random.choice(['eggs', 'meat'], size=6)
index = pd.MultiIndex.from_arrays([colors, foods], names=['color', 'food'])
df = pd.DataFrame(np.arange(12).reshape(6, 2), index=index)

# 实:4.1：索引名
df.query('color == "red"')
 
# 实例4.2：索引无名称
df.index.names = [None, None]
df.query('ilevel_0 == "red"') #ilevel_0第0级的索引级别
df.query('ilevel_1 == "meat"')#ilevel_1第1级的索引级别

,,0,1
red,meat,8,9


#### 实例5

In [78]:

# 实例6：Python与pandas语法比较
# 完全类似numpy的语法
 
# 实例6.1：比较运算符，逻辑运算符
df = pd.DataFrame(np.random.randint(10, size=(10, 3)), columns=list('ABC'))
 
df.query('(A< B) & (B< C)')
df[(df.A < df.B) & (df.B < df.C)]
df.query('A< B & B < C')
df.query('A< B and B < C')
df.query('A < B < C') #全部等价
 
#============================================================
# 实例6.2：==操作符与list对象的特殊用法
# ==/ !=工程，以类似in/not in
df = pd.DataFrame({'a': list('aabbccddeeff'), 'b': list('aaaabbbbcccc'),
                   'c': np.random.randint(5, size=12),
                   'd': np.random.randint(9, size=12)})
df.query('b == ["a", "b", "c"]')==df[df.b.isin(["a", "b", "c"])]
 
df.query('c == [1, 2]')
df.query('c != [1, 2]')
 
# using in/not in
df.query('[1, 2] in c')
df.query('[1, 2] not in c')
df[df.c.isin([1, 2])]# pure Python

#============================================================
# 实例6.3：in与not in
df = pd.DataFrame({'a': list('abcdef'), 'b': list('fedfed'),'c': 5, 'd':5})
 

 
df.query('a in b and c < d') #与其他表达式结合获得非常简洁查询
df[df.b.isin(df.a) & (df.c < df.d)]
 
result1=df[df.a.isin(df.b)]
result2=df.query('a not in b')
result3=df[~df.a.isin(df.b)] # pure Python
 
#============================================================
# 实例6.4：布尔运算符not或~运算符否定布尔表达式
 
df = pd.DataFrame(np.arange(9).reshape(3,3), columns=list('ABC'))
df['bools'] = df.eval('C>=5')
 
result1=df.query('not bools')
result2=(df.query('not bools') == df[~df.bools])
 

# 复杂表达式：
df.query('A < B< C and (not bools) or bools > 2')               #短查询语法
df[(df.A < df.B) & (df.B < df.C) & (~df.bools) | (df.bools > 2)]#等效于纯Python

,A,B,C,bools
0,0,1,2,False
